# Features Extracting using LBP and GLCM

Libraries and Modules

In [ ]:
import os
import numpy as np
import pandas as pd
from skimage import io, color, img_as_ubyte
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops

The code uses an augmented dataset created by augmentation forces, saved in defect list files. Extracted features will be saved in a .csv file containing values describing features extracted by GLCM and LBP to train the model.

In [ ]:
# Define the GLCM parameters
glcm_params = [(1, 0), (1, np.pi/4), (1, np.pi/2), (1, 3*np.pi/4)]

# Define the LBP parameters
lbp_params = [(1, 8), (2, 8), (3, 8)]

# Define the minimum and maximum values for normalization
min_contrast = 0.0
max_contrast = 1000.0
min_energy = 0.0
max_energy = 1.0
min_entropy = 0.0
max_entropy = 10.0  
min_homogeneity = 0.0
max_homogeneity = 1.0
min_correlation = -1.0
max_correlation = 1.0
min_dissimilarity = 0.0  
max_dissimilarity = 100.0  
min_asm = 0.00001  
max_asm = 0.05  

# Initialize lists to store normalized results
image_names = []
normalized_features = []
defect_types = []

# List of defect categories (folder names)
defect_categories = [
 # list of Defects 
]

# Loop through the defect categories
for category in defect_categories:
    # Get a list of image filenames in the current category folder
    image_filenames = os.listdir(category)

    # Iterate through all image files in the folder
    for image_filename in image_filenames:
        if image_filename.endswith('.jpg'):
            # Load the image
            image = io.imread(os.path.join(category, image_filename))
            
            # Convert the image to grayscale
            gray_image = color.rgb2gray(image)

            # Convert the grayscale image to uint8
            gray_image = img_as_ubyte(gray_image)

            # Initialize a list to store features for this image
            features = []

            # Calculate GLCM features
            for distance, angle in glcm_params:
                # Calculate the GLCM
                glcm = graycomatrix(gray_image, [distance], [angle], levels=256)

                # Normalize the GLCM
                glcm_normalized = glcm.astype(np.float32) / np.sum(glcm)

                # Calculate texture properties
                contrast = graycoprops(glcm_normalized, 'contrast')[0, 0]
                energy = graycoprops(glcm_normalized, 'energy')[0, 0]
                entropy = -np.sum(glcm_normalized * np.log(glcm_normalized + np.finfo(float).eps))
                homogeneity = graycoprops(glcm_normalized, 'homogeneity')[0, 0]
                correlation = graycoprops(glcm_normalized, 'correlation')[0, 0]
                dissimilarity = graycoprops(glcm_normalized, 'dissimilarity')[0, 0]
                asm = graycoprops(glcm_normalized, 'ASM')[0, 0]

                # Normalize the texture characteristics
                normalized_contrast = (contrast - min_contrast) / (max_contrast - min_contrast)
                normalized_energy = (energy - min_energy) / (max_energy - min_energy)
                # Convert ASM to string, remove leading zeros, and convert back to float
                normalized_entropy = float(str(entropy).lstrip('0'))
                normalized_entropy = (normalized_entropy - min_entropy) / (max_entropy - min_entropy)
                normalized_homogeneity = (homogeneity - min_homogeneity) / (max_homogeneity - min_homogeneity)
                normalized_correlation = (correlation - min_correlation) / (max_correlation - min_correlation)
                # Adjusted min and max values for dissimilarity and ASM
                normalized_dissimilarity = (dissimilarity - min_dissimilarity) / (max_dissimilarity - min_dissimilarity)
                # Convert ASM to string, remove leading zeros, and convert back to float
                normalized_asm = float(str(asm).lstrip('0'))
                normalized_asm = (normalized_asm - min_asm) / (max_asm - min_asm)

                # Append the normalized results to the features list
                features.extend([normalized_contrast, normalized_energy, normalized_entropy, normalized_homogeneity, normalized_correlation, normalized_dissimilarity, normalized_asm])

            # Calculate LBP features
            for lbp_radius, lbp_n_points in lbp_params:
                # Calculate LBP features
                lbp_image = local_binary_pattern(gray_image, lbp_n_points, lbp_radius, method='uniform')

                # Calculate LBP histogram as features
                lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, lbp_n_points + 3), range=(0, lbp_n_points + 2))

                # Normalize the histogram
                lbp_hist = lbp_hist.astype(np.float32)
                lbp_hist /= (lbp_hist.sum() + 1e-6)

                # Add LBP features for the current parameter combination
                features.extend(lbp_hist)
            
            # Append the features for this image to the result
            normalized_features.append(features)

            # Append the defect type
            defect_types.append(category)

# Create a DataFrame to store the data
column_names = []

# Add column names for GLCM features
for distance, angle in glcm_params:
    for char in ['Contrast', 'Energy', 'Entropy', 'Homogeneity', 'Correlation', 'Dissimilarity', 'ASM']:
        column_names.append(f' {char} (d={distance}, a={angle:.2f})')

# Add column names for LBP features
for lbp_radius, lbp_n_points in lbp_params:
    column_names.extend([f'LBP_{lbp_radius}_{lbp_n_points}_{i}' for i in range(lbp_n_points + 2)])

# Create the DataFrame
df = pd.DataFrame(normalized_features, columns=column_names)

# Add the defect type column
df['Defect'] = defect_types

# Save the results in a CSV file
output_filename = 'lbp_and_glcm_features.csv'
df.to_csv(output_filename, index=False)

print(f'Results saved to {output_filename}')


# Checking Features in .csv file

In [ ]:
import pandas as pd

# Specify the path to your CSV file
csv_file_path = "lbp_and_glcm_features.csv"

# Use pandas to read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Number of rows and columns
num_rows, num_columns = df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

# Column names
column_names = df.columns
print("Column names:", column_names)

# View the first few rows
print("First few rows:")
print(df.head())

# View the last few rows
print("Last few rows:")
print(df.tail())

# Summary statistics
print("Summary statistics:")
print(df.describe())

# If you want to count non-null values in each column, you can use:
print(df.count())